In [ ]:
# Cell 1: Imports
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torchmetrics import Accuracy, Precision, Recall, F1Score, JaccardIndex
from torchmetrics.collections import MetricCollection
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import random
from terratorch.tasks import SemanticSegmentationTask
import rasterio
import glob
import re
import traceback

# Cell 2: Configuration 
BASE_DATA_DIR_ACTUAL = "<DATA_DIR>"
CHECKPOINT_SAVE_DIR_ACTUAL = "<CHECKPOINT_DIR>"
LOGS_DIR_ACTUAL = "<LOGS_DIR>"

PATCHES_DIR = os.path.join(BASE_DATA_DIR_ACTUAL, "GEE_WeeklyPatches")
TARGETS_DIR = os.path.join(BASE_DATA_DIR_ACTUAL, "GEE_WeeklyMasks")

os.makedirs(CHECKPOINT_SAVE_DIR_ACTUAL, exist_ok=True)
os.makedirs(LOGS_DIR_ACTUAL, exist_ok=True)

EXPECTED_BANDS_ORDER = ["B02", "B03", "B04", "B8A", "B11", "B12"]
NUM_CLASSES = 2
IMG_SIZE = 256

BATCH_SIZE = 4
EPOCHS = 100
LR = 1e-5
WEIGHT_DECAY = 0.01
HEAD_DROPOUT = 0.2
FREEZE_BACKBONE = False

BRIGHTNESS_RANGE = (0.95, 1.05)
CONTRAST_RANGE = (0.95, 1.05)
GAMMA_RANGE = (0.95, 1.05)
GAUSSIAN_NOISE_STD = 0.0
AUG_PROBABILITY = 0.3

EARLY_STOPPING_PATIENCE = 15
EARLY_STOPPING_MONITOR = "val_loss"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"PyTorch is available: {torch.cuda.is_available()} (CUDA)")
if DEVICE == "cuda":
    print(f"CUDA available: {torch.cuda.device_count()} device(s) detected.")

# Cell 3: Custom Dataset Definition 
class CerealSensingDataset(Dataset):
    def __init__(self, file_identifiers, patches_dir, targets_dir, target_img_size, 
                 augmentations=False, 
                 brightness_range=(1.0, 1.0), 
                 contrast_range=(1.0, 1.0),   
                 gamma_range=(1.0, 1.0),      
                 gaussian_noise_std=0.0,      
                 aug_probability=0.5):
        self.file_identifiers = file_identifiers
        self.patches_dir = patches_dir
        self.targets_dir = targets_dir
        self.target_img_size = target_img_size
        self.augmentations = augmentations
        self.brightness_range = brightness_range
        self.contrast_range = contrast_range
        self.gamma_range = gamma_range
        self.gaussian_noise_std = gaussian_noise_std
        self.aug_probability = aug_probability

    def __len__(self):
        return len(self.file_identifiers)

    def __getitem__(self, idx):
        identifier = self.file_identifiers[idx]
        image_filename = f"{identifier}_S2.tif"
        mask_filename = f"{identifier}_Mask.tif"
        image_path = os.path.join(self.patches_dir, image_filename)
        label_path = os.path.join(self.targets_dir, mask_filename)

        try:
            with rasterio.open(image_path) as src:
                image = src.read()
                if image.shape[0] != 6:
                    raise ValueError(f"Image {identifier} has {image.shape[0]} bands, expected 6.")
            with rasterio.open(label_path) as src:
                label = src.read(1)

            image_tensor = torch.tensor(image, dtype=torch.float32)
            label_tensor_float = torch.tensor(label, dtype=torch.float32)

            image_tensor = F.interpolate(image_tensor.unsqueeze(0),
                                         size=(self.target_img_size, self.target_img_size),
                                         mode='bilinear', align_corners=False).squeeze(0)
            label_tensor_float = F.interpolate(label_tensor_float.unsqueeze(0).unsqueeze(0),
                                         size=(self.target_img_size, self.target_img_size),
                                         mode='nearest').squeeze(0).squeeze(0)

            if self.augmentations:
                num_channels = image_tensor.shape[0]

                if random.random() < self.aug_probability:
                    channel_to_aug = random.randint(0, num_channels - 1)
                    brightness_factor = random.uniform(self.brightness_range[0], self.brightness_range[1])
                    image_tensor[channel_to_aug:channel_to_aug+1] = TF.adjust_brightness(
                        image_tensor[channel_to_aug:channel_to_aug+1], brightness_factor
                    )

                if random.random() < self.aug_probability:
                    channel_to_aug = random.randint(0, num_channels - 1)
                    contrast_factor = random.uniform(self.contrast_range[0], self.contrast_range[1])
                    image_tensor[channel_to_aug:channel_to_aug+1] = TF.adjust_contrast(
                        image_tensor[channel_to_aug:channel_to_aug+1], contrast_factor
                    )

                if random.random() < self.aug_probability:
                    channel_to_aug = random.randint(0, num_channels - 1)
                    gamma = random.uniform(self.gamma_range[0], self.gamma_range[1])
                    gamma = max(0.1, gamma)
                    channel_non_neg = torch.clamp(image_tensor[channel_to_aug:channel_to_aug+1], min=0)
                    image_tensor[channel_to_aug:channel_to_aug+1] = TF.adjust_gamma(
                        channel_non_neg, gamma
                    )

                if self.gaussian_noise_std > 0 and random.random() < self.aug_probability:
                    channel_to_aug = random.randint(0, num_channels - 1)
                    noise = torch.randn_like(image_tensor[channel_to_aug]) * self.gaussian_noise_std
                    image_tensor[channel_to_aug] = image_tensor[channel_to_aug] + noise

                image_tensor = image_tensor.float()

            label_tensor = label_tensor_float.long()

            if image_tensor.shape != (6, self.target_img_size, self.target_img_size) or \
               label_tensor.shape != (self.target_img_size, self.target_img_size):
                raise ValueError(f"Shape mismatch for {identifier} post-processing. Img: {image_tensor.shape}, Lbl: {label_tensor.shape}")
            return image_tensor, label_tensor
        except Exception as e:
            print(f"Error loading/processing {identifier}: {e}")
            traceback.print_exc()
            raise e

# Cell 4: Data Discovery, Integrity Check, Splitting and Loaders 
print("\n=========== Data Loading & Prep ============")
image_files_pattern = os.path.join(PATCHES_DIR, "Patch*_S2.tif")
all_image_filepaths = sorted(glob.glob(image_files_pattern))
initial_file_identifiers = []
if not all_image_filepaths: print(f"ERROR: No image files found in {PATCHES_DIR}")
else: print(f"Found {len(all_image_filepaths)} potential image files. Checking masks...")
for img_path in all_image_filepaths:
    base_name = os.path.basename(img_path)
    identifier_match = re.match(r"(Patch\d+_W\d+_\d{8})_S2\.tif", base_name)
    if identifier_match:
        identifier = identifier_match.group(1)
        mask_path = os.path.join(TARGETS_DIR, f"{identifier}_Mask.tif")
        if os.path.exists(mask_path): initial_file_identifiers.append(identifier)
        else: print(f"WARNING: Mask for {identifier} not found. Skipping.")
    else: print(f"WARNING: Could not parse {base_name}. Skipping.")
print(f"Found {len(initial_file_identifiers)} initial image-mask pairs.")

print(f"\nPerforming data integrity check on {len(initial_file_identifiers)} pairs...")
corrupted_identifiers, valid_identifiers = [], []
if not initial_file_identifiers: print("No identifiers to check.")
else:
    for identifier in initial_file_identifiers:
        image_path = os.path.join(PATCHES_DIR, f"{identifier}_S2.tif")
        try:
            with rasterio.open(image_path) as src: image_data = src.read()
            problem, msgs = False, []
            if np.isnan(image_data).any(): msgs.append("NaNs"); problem = True
            if image_data.max() > 65535 or image_data.min() < -32768 : 
                msgs.append(f"Values out of typical sensor range (min: {image_data.min()}, max: {image_data.max()})"); problem = True
            if image_data.size > 0:
                is_uniform_check = True
                for band_idx_check in range(image_data.shape[0]):
                    first_val_band_check = image_data[band_idx_check,0,0] if image_data.shape[1] > 0 and image_data.shape[2] > 0 else None
                    if first_val_band_check is not None:
                        if np.issubdtype(image_data[band_idx_check].dtype, np.floating):
                            if not np.allclose(image_data[band_idx_check], first_val_band_check, atol=1e-5):
                                is_uniform_check = False; break
                        else:
                            if not np.all(image_data[band_idx_check] == first_val_band_check):
                                is_uniform_check = False; break
                    elif image_data.shape[1] > 0 and image_data.shape[2] > 0 : 
                        is_uniform_check = False; break 
                if is_uniform_check and image_data.shape[1] > 0 and image_data.shape[2] > 0:
                     msgs.append(f"Uniform ({image_data[0,0,0]})"); problem = True
            else: msgs.append("Empty data"); problem = True
            if problem: print(f"WARNING: {identifier} issues: {'; '.join(msgs)}. Skipping."); corrupted_identifiers.append(identifier)
            else: valid_identifiers.append(identifier)
        except Exception as e: print(f"ERROR checking {identifier}: {e}. Skipping."); corrupted_identifiers.append(identifier)
print(f"Integrity check complete. Corrupted: {len(corrupted_identifiers)}.")
clean_identifiers = valid_identifiers
num_clean_patches = len(clean_identifiers)
print(f"Clean patches: {num_clean_patches}")

train_ds, val_ds, train_loader, val_loader = None, None, None, None
if num_clean_patches == 0: print("ERROR: No clean patches available.")
else:
    random.shuffle(clean_identifiers)
    if num_clean_patches >= 2:
        num_train = max(1, int(num_clean_patches * 0.8))
        train_ids, val_ids = clean_identifiers[:num_train], clean_identifiers[num_train:]
        if not val_ids and len(train_ids) > 1: val_ids = [train_ids.pop()]
        elif not val_ids and len(train_ids) == 1: print("WARNING: Only one sample available, cannot create val split.")
    elif num_clean_patches == 1:
        train_ids, val_ids = clean_identifiers, []
        print("WARNING: Only one sample. Training set has 1, validation is empty.")
    else: train_ids, val_ids = [], []
    print(f"Training IDs: {len(train_ids)}, Validation IDs: {len(val_ids)}")

    if train_ids:
        train_ds = CerealSensingDataset(
            train_ids, PATCHES_DIR, TARGETS_DIR, IMG_SIZE, 
            augmentations=True, 
            brightness_range=BRIGHTNESS_RANGE,
            contrast_range=CONTRAST_RANGE,
            gamma_range=GAMMA_RANGE,
            gaussian_noise_std=GAUSSIAN_NOISE_STD,
            aug_probability=AUG_PROBABILITY
        )
        num_data_workers = 4 
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_data_workers, pin_memory=True if DEVICE=="cuda" else False, persistent_workers=True if num_data_workers > 0 else False)
    else: print("ERROR: Training dataset empty.")
    if val_ids:
        val_ds = CerealSensingDataset(val_ids, PATCHES_DIR, TARGETS_DIR, IMG_SIZE, augmentations=False) 
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_data_workers, pin_memory=True if DEVICE=="cuda" else False, persistent_workers=True if num_data_workers > 0 else False)
    else: print("WARNING: Validation dataset empty.")
    if train_loader: print(f"Train DataLoader created with {len(train_loader.dataset)} samples. Augmentations: True (Radiometric Only)")
    if val_loader: print(f"Val DataLoader created with {len(val_loader.dataset)} samples. Augmentations: False")
    else: val_loader = None

# Cell 5: Quick Visualisation (Sanity Check)
if train_ds and len(train_ds) > 0:
    try:
        print("\n--- Visualizing a sample from TRAIN_DS (potentially radiometrically augmented) ---")
        sample_img_aug, sample_mask_aug = train_ds[1] 
        print(f"Augmented Sample Img Shape: {sample_img_aug.shape}, Mask Shape: {sample_mask_aug.shape}, Mask Unique: {torch.unique(sample_mask_aug)}")
        img_display_aug = sample_img_aug[:3].cpu().numpy()
        img_min_aug, img_max_aug = img_display_aug.min(axis=(1,2),keepdims=True), img_display_aug.max(axis=(1,2),keepdims=True)
        denominator_aug = img_max_aug - img_min_aug
        denominator_aug[denominator_aug < 1e-6] = 1e-6
        img_norm_aug = np.clip((img_display_aug - img_min_aug) / denominator_aug, 0, 1).transpose(1,2,0)
        
        fig_aug, axs_aug = plt.subplots(1,2, figsize=(10,5))
        axs_aug[0].imshow(img_norm_aug); axs_aug[0].set_title(f'Augmented Input (ID: {train_ds.file_identifiers[0]})'); axs_aug[0].axis('off')
        axs_aug[1].imshow(sample_mask_aug.cpu().numpy(), cmap='gray'); axs_aug[1].set_title('Corresponding Mask'); axs_aug[1].axis('off')
        plt.tight_layout(); plt.show()
    except Exception as e: print(f"Visualization error for augmented sample: {e}")
else: print("Training dataset empty or not initialized. Skipping augmented sample visualization.")

# Cell 6: Load TerraTorch Model Configuration
model_args = {
    "backbone": "prithvi_eo_v2_300", "backbone_pretrained": True,
    "backbone_bands": EXPECTED_BANDS_ORDER, "decoder": "FCNDecoder",
    "decoder_channels": 256, "head_dropout": HEAD_DROPOUT,
    "num_classes": NUM_CLASSES, "rescale": True
}
task_config_obj = SemanticSegmentationTask(
    model_args=model_args, model_factory="EncoderDecoderFactory", loss="ce",
    freeze_backbone=FREEZE_BACKBONE, lr=LR, optimizer="adamw",
    optimizer_hparams={"weight_decay": WEIGHT_DECAY}
)
print(f"Backbone freezing: {FREEZE_BACKBONE}. Model configured for {len(EXPECTED_BANDS_ORDER)} bands.")

# Cell 7: PyTorch Lightning Module Definition
class LitSegmentationModel(LightningModule):
    def __init__(self, model_arch, learning_rate, weight_decay, num_classes, class_names=None):
        super().__init__()
        self.save_hyperparameters(ignore=['model_arch'])
        self.model = model_arch
        self.class_weights_tensor = None 
        self.class_names = class_names if class_names else [f"class{i}" for i in range(num_classes)]

        metrics = MetricCollection({
            'accuracy_overall': Accuracy(task="multiclass", num_classes=self.hparams.num_classes, average='micro'),
            'precision_macro': Precision(task="multiclass", num_classes=self.hparams.num_classes, average='macro'),
            'recall_macro': Recall(task="multiclass", num_classes=self.hparams.num_classes, average='macro'),
            'f1_score_macro': F1Score(task="multiclass", num_classes=self.hparams.num_classes, average='macro'),
            'jaccard_index_per_class': JaccardIndex(task="multiclass", num_classes=self.hparams.num_classes, average='none'),
            'accuracy_per_class': Accuracy(task="multiclass", num_classes=self.hparams.num_classes, average='none'),
            'precision_per_class': Precision(task="multiclass", num_classes=self.hparams.num_classes, average='none'),
            'recall_per_class': Recall(task="multiclass", num_classes=self.hparams.num_classes, average='none'),
            'f1_score_per_class': F1Score(task="multiclass", num_classes=self.hparams.num_classes, average='none'),
        })
        self.val_metrics = metrics.clone(prefix='val_')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        out_struct = self.model(x)
        logits = out_struct.output
        weights = self.class_weights_tensor.to(logits.device) if self.class_weights_tensor is not None else None
        loss = F.cross_entropy(logits, y, weight=weights)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out_struct = self.model(x)
        logits = out_struct.output
        weights = self.class_weights_tensor.to(logits.device) if self.class_weights_tensor is not None else None
        loss = F.cross_entropy(logits, y, weight=weights)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True, sync_dist=True) 
        preds_classes = torch.argmax(logits, dim=1)
        self.val_metrics.update(preds_classes, y)
        return loss

    def on_validation_epoch_end(self):
        metrics_output = self.val_metrics.compute()
        for name, value in metrics_output.items():
            log_to_prog_bar = False 
            prog_bar_name_override = None
            if name in ["val_accuracy_overall", "val_f1_score_macro"]: log_to_prog_bar = True
            if isinstance(value, torch.Tensor) and value.numel() == self.hparams.num_classes:
                for i in range(self.hparams.num_classes):
                    class_metric_name_for_csv = f"{name}_{self.class_names[i]}"
                    current_metric_prog_bar_status = False
                    if self.class_names[i] == "cereal": 
                        if name == "val_jaccard_index_per_class": prog_bar_name_override = "val_iou_cereal"; current_metric_prog_bar_status = True
                        elif name == "val_f1_score_per_class": prog_bar_name_override = "val_f1_cereal"; current_metric_prog_bar_status = True
                    if prog_bar_name_override and current_metric_prog_bar_status:
                        self.log(prog_bar_name_override, value[i], prog_bar=True, logger=False, on_step=False, on_epoch=True, sync_dist=True)
                        self.log(class_metric_name_for_csv, value[i], prog_bar=False, logger=True, on_step=False, on_epoch=True, sync_dist=True)
                        prog_bar_name_override = None 
                    else:
                        self.log(class_metric_name_for_csv, value[i], prog_bar=False, logger=True, on_step=False, on_epoch=True, sync_dist=True)
            else: 
                if not (prog_bar_name_override and name.endswith(prog_bar_name_override.split('_')[-1])): 
                    self.log(name, value, prog_bar=log_to_prog_bar, logger=True, on_step=False, on_epoch=True, sync_dist=True)
        self.val_metrics.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=self.hparams.weight_decay)
        return optimizer

CLASS_NAMES_FOR_LOGGING = ["non_cereal", "cereal"] 
lightning_model = None
if 'task_config_obj' in locals() or 'task_config_obj' in globals():
    lightning_model = LitSegmentationModel(
        model_arch=task_config_obj.model, learning_rate=LR, weight_decay=WEIGHT_DECAY,
        num_classes=NUM_CLASSES, class_names=CLASS_NAMES_FOR_LOGGING
    )
    print("LitSegmentationModel instantiated.")
else: print("ERROR: task_config_obj not defined.")

# Cell 8: Training
csv_logger = CSVLogger(save_dir=LOGS_DIR_ACTUAL, name="cereal_segmentation_only_rad_aug_run")
print(f"CSV logs will be saved in: {csv_logger.log_dir}")

loss_checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINT_SAVE_DIR_ACTUAL, filename='model-only-rad-aug-best-loss-{epoch:02d}-{val_loss:.3f}',
    save_top_k=1, verbose=True, monitor='val_loss', mode='min'
)
iou_cereal_checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINT_SAVE_DIR_ACTUAL, filename='model-only-rad-aug-best-iou-cereal-{epoch:02d}-{val_jaccard_index_per_class_cereal:.3f}',
    save_top_k=1, verbose=True, monitor='val_jaccard_index_per_class_cereal', mode='max'
)
early_stop_callback = EarlyStopping(
    monitor=EARLY_STOPPING_MONITOR, patience=EARLY_STOPPING_PATIENCE, verbose=True,
    mode="min" if "loss" in EARLY_STOPPING_MONITOR.lower() else "max"
)

if lightning_model and train_loader: 
    is_interactive = False
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell': is_interactive = True
    except NameError: is_interactive = False

    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        strategy_to_use = "ddp_notebook" if is_interactive else "ddp_find_unused_parameters_true"
        print(f"INFO: Multiple GPUs detected. Using '{strategy_to_use}' strategy.")
    else:
        strategy_to_use = "auto"
        print(f"INFO: Single GPU or CPU. Using '{strategy_to_use}' strategy.")

    trainer = Trainer(
        accelerator="gpu" if DEVICE == "cuda" else "cpu",
        devices="auto" if DEVICE == "cuda" else 1, 
        strategy=strategy_to_use, 
        num_nodes=1, max_epochs=EPOCHS, log_every_n_steps=10, logger=csv_logger,    
        callbacks=[early_stop_callback, loss_checkpoint_callback, iou_cereal_checkpoint_callback],
        gradient_clip_val=0.5,
    )
    print("Starting training with ONLY radiometric augmentations...")
    try:
        val_dataloaders_arg = val_loader if val_loader and len(val_loader.dataset) > 0 else None
        if not val_dataloaders_arg and EARLY_STOPPING_MONITOR.startswith("val_"):
            print(f"WARNING: val_loader is None/empty, but EarlyStopping monitors '{EARLY_STOPPING_MONITOR}'. Adjusting callbacks.")
            callbacks_list = [c for c in [loss_checkpoint_callback, iou_cereal_checkpoint_callback] if c.monitor is None or not c.monitor.startswith("val_")]
            if not EARLY_STOPPING_MONITOR.startswith("val_"): callbacks_list.append(early_stop_callback)
            else: print("EarlyStopping on validation metric removed as val_loader is absent.")
            trainer.callbacks = callbacks_list

        trainer.fit(lightning_model, train_dataloaders=train_loader, val_dataloaders=val_dataloaders_arg)
        print("Training finished.")
        if loss_checkpoint_callback.best_model_path: print(f"Best model (by loss) saved at: {loss_checkpoint_callback.best_model_path}")
        if iou_cereal_checkpoint_callback.best_model_path: print(f"Best model (by IoU for cereal) saved at: {iou_cereal_checkpoint_callback.best_model_path}")
    except Exception as e:
        print(f"An error occurred during training: {e}"); traceback.print_exc()
else:
    print("ERROR: Training prerequisites not met (model or train_loader is None).")
